In [3]:
import os
import json
import requests
import joblib
import pickle
import pandas as pd
import numpy as np 

from statistics import mode
import plotly.express as px

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
#from catboost import CatBoostClassifier

import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler


In [4]:
pd.options.plotting.backend = "plotly"

# Data and Descriptive Stats 

In [32]:
train=pd.read_csv("./dataset/train.csv")
train.head()

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,CST_115179,ita Bose,46,F,N,Y,0.0,107934.04,612.0,Unknown,1.0,1.0,33070.28,18690.93,73,544.0,2,1,1
1,CST_121920,Alper Jonathan,29,M,N,Y,0.0,109862.62,2771.0,Laborers,2.0,0.0,15329.53,37745.19,52,857.0,0,0,0
2,CST_109330,Umesh Desai,37,M,N,Y,0.0,230153.17,204.0,Laborers,2.0,0.0,48416.60,41598.36,43,650.0,0,0,0
3,CST_128288,Rie,39,F,N,Y,0.0,122325.82,11941.0,Core staff,2.0,0.0,22574.36,32627.76,20,754.0,0,0,0
4,CST_151355,McCool,46,M,Y,Y,0.0,387286.00,1459.0,Core staff,1.0,0.0,38282.95,52950.64,75,927.0,0,0,0


In [6]:
test=pd.read_csv("./dataset/test.csv")
test.head()

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months
0,CST_142525,Siva,52,F,Y,N,0.0,232640.53,998.0,Unknown,2.0,0.0,14406.73,26524.40,4,779.0,0,0
1,CST_129215,Scott,48,F,N,N,1.0,284396.79,1338.0,Unknown,3.0,0.0,57479.99,68998.72,70,806.0,0,0
2,CST_138443,Victoria,50,F,N,N,1.0,149419.28,1210.0,Unknown,3.0,0.0,21611.01,25187.80,71,528.0,2,0
3,CST_123812,John McCrank,30,F,N,N,1.0,160437.54,503.0,Laborers,2.0,1.0,28990.76,29179.39,9,815.0,0,0
4,CST_144450,Martinne,52,M,N,Y,0.0,233480.37,157.0,Laborers,2.0,1.0,54213.72,82331.82,82,613.0,1,1


In [7]:
train.shape

(45528, 19)

In [8]:
test.shape

(11383, 18)

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45528 entries, 0 to 45527
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              45528 non-null  object 
 1   name                     45528 non-null  object 
 2   age                      45528 non-null  int64  
 3   gender                   45528 non-null  object 
 4   owns_car                 44981 non-null  object 
 5   owns_house               45528 non-null  object 
 6   no_of_children           44754 non-null  float64
 7   net_yearly_income        45528 non-null  float64
 8   no_of_days_employed      45065 non-null  float64
 9   occupation_type          45528 non-null  object 
 10  total_family_members     45445 non-null  float64
 11  migrant_worker           45441 non-null  float64
 12  yearly_debt_payments     45433 non-null  float64
 13  credit_limit             45528 non-null  float64
 14  credit_limit_used(%)  

In [10]:
train.describe()

,age,no_of_children,net_yearly_income,no_of_days_employed,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
count,45528.000000,44754.000000,4.552800e+04,45065.000000,45445.000000,45441.000000,45433.000000,4.552800e+04,45528.00000,45520.000000,45528.000000,45528.000000,45528.000000
mean,38.993411,0.420655,2.006556e+05,67609.289293,2.158081,0.179111,31796.965311,4.354842e+04,52.23502,782.791257,0.060710,0.050540,0.081203
std,9.543990,0.724097,6.690740e+05,139323.524434,0.911572,0.383450,17269.727234,1.487847e+05,29.37691,100.619746,0.264629,0.219059,0.273149
min,23.000000,0.000000,2.717061e+04,2.000000,1.000000,0.000000,2237.470000,4.003140e+03,0.00000,500.000000,0.000000,0.000000,0.000000
25%,31.000000,0.000000,1.263458e+05,936.000000,2.000000,0.000000,19231.140000,2.397381e+04,27.00000,704.000000,0.000000,0.000000,0.000000
50%,39.000000,0.000000,1.717149e+05,2224.000000,2.000000,0.000000,29081.650000,3.568804e+04,54.00000,786.000000,0.000000,0.000000,0.000000
75%,47.000000,1.000000,2.406038e+05,5817.000000,3.000000,0.000000,40561.150000,5.343576e+04,78.00000,867.000000,0.000000,0.000000,0.000000
max,55.000000,9.000000,1.407590e+08,365252.000000,10.000000,1.000000,328112.860000,3.112997e+07,99.00000,949.000000,2.000000,1.000000,1.000000


In [11]:
train.corr()

,age,no_of_children,net_yearly_income,no_of_days_employed,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
age,1.000000,-0.009082,0.004079,0.000506,-0.010640,-0.005519,-0.002453,0.004473,-0.005529,0.000817,0.001393,-0.001406,-0.000983
no_of_children,-0.009082,1.000000,0.008939,-0.243088,0.880278,0.070673,0.028706,0.009354,0.007487,-0.014673,0.019528,0.021620,0.023689
net_yearly_income,0.004079,0.008939,1.000000,-0.029367,0.010432,0.001478,0.076958,0.993378,0.002696,-0.009688,-0.004696,0.015092,0.011508
no_of_days_employed,0.000506,-0.243088,-0.029367,1.000000,-0.229344,-0.219521,-0.104545,-0.027745,-0.018720,0.035533,-0.036232,-0.034295,-0.047043
total_family_members,-0.010640,0.880278,0.010432,-0.229344,1.000000,0.080634,0.078669,0.010251,0.001928,-0.011524,0.010608,0.013411,0.010754
migrant_worker,-0.005519,0.070673,0.001478,-0.219521,0.080634,1.000000,0.014170,-0.000110,0.010198,-0.013319,0.029202,0.029964,0.034013
yearly_debt_payments,-0.002453,0.028706,0.076958,-0.104545,0.078669,0.014170,1.000000,0.075169,-0.006850,0.005756,-0.012738,-0.011619,-0.013089
credit_limit,0.004473,0.009354,0.993378,-0.027745,0.010251,-0.000110,0.075169,1.000000,0.003109,-0.009685,-0.004302,0.015759,0.012250
credit_limit_used(%),-0.005529,0.007487,0.002696,-0.018720,0.001928,0.010198,-0.006850,0.003109,1.000000,-0.174420,0.252505,0.253684,0.326641
credit_score,0.000817,-0.014673,-0.009688,0.035533,-0.011524,-0.013319,0.005756,-0.009685,-0.174420,1.000000,-0.471421,-0.450712,-0.543164


#### Shape of Data :
Train : (45528, 19) <br>
Test :  (11383, 18)


#### Numerical Features : 
['age', 'no_of_children', 'net_yearly_income', 'no_of_days_employed',
       'total_family_members', 'migrant_worker', 'yearly_debt_payments',
       'credit_limit', 'credit_limit_used(%)', 'credit_score', 'prev_defaults',
       'default_in_last_6months']

#### Categorical Features : 
['owns_car','gender','owns_house','occupation_type']

#### Target :
credit_card_default

In [12]:
train['credit_card_default'] = train.credit_card_default.astype('category')

# Looking at Distribution  of Target with Catgorical Features

In [13]:
train[train["owns_car"]=='Y']["credit_card_default"].value_counts()

0    14101
1     1137
Name: credit_card_default, dtype: int64

In [14]:
train[train["owns_car"]=='N']["credit_card_default"].value_counts()

0    27229
1     2514
Name: credit_card_default, dtype: int64

**Customer owning car is a not a big indicator if the credit will deafault**

In [15]:
train[train["gender"]=='M']["credit_card_default"].value_counts()

0    13966
1     1604
Name: credit_card_default, dtype: int64

In [16]:
train[train["gender"]=='F']["credit_card_default"].value_counts()

0    27864
1     2093
Name: credit_card_default, dtype: int64

**Customer Gender is also not a big indicator if the credit will deafault**

In [73]:
train[train["owns_house"]=='Y']["credit_card_default"].value_counts()

0    29088
1     2554
Name: credit_card_default, dtype: int64

In [74]:
train[train["owns_house"]=='N']["credit_card_default"].value_counts()

0    12743
1     1143
Name: credit_card_default, dtype: int64

**Customer Owning an house is also not a big indicator if the credit will deafault**

In [111]:
train["occupation_type"].unique()


array(['Unknown', 'Laborers', 'Core staff', 'Accountants',
       'High skill tech staff', 'Sales staff', 'Managers', 'Drivers',
       'Medicine staff', 'Cleaning staff', 'HR staff', 'Security staff',
       'Cooking staff', 'Waiters/barmen staff', 'Low-skill Laborers',
       'Private service staff', 'Secretaries', 'Realty agents',
       'IT staff'], dtype=object)

A graph to plot deafults according to occupation

In [104]:
#fig = px.bar(train,x="credit_card_default")
#fig.show()

# Removing gender and owns_house 

In [34]:
train1=train.drop(["gender","owns_house","owns_car"],axis=1)

In [35]:
train1.head()

,customer_id,name,age,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,CST_115179,ita Bose,46,0.0,107934.04,612.0,Unknown,1.0,1.0,33070.28,18690.93,73,544.0,2,1,1
1,CST_121920,Alper Jonathan,29,0.0,109862.62,2771.0,Laborers,2.0,0.0,15329.53,37745.19,52,857.0,0,0,0
2,CST_109330,Umesh Desai,37,0.0,230153.17,204.0,Laborers,2.0,0.0,48416.60,41598.36,43,650.0,0,0,0
3,CST_128288,Rie,39,0.0,122325.82,11941.0,Core staff,2.0,0.0,22574.36,32627.76,20,754.0,0,0,0
4,CST_151355,McCool,46,0.0,387286.00,1459.0,Core staff,1.0,0.0,38282.95,52950.64,75,927.0,0,0,0


In [36]:
X1=train1.drop(["customer_id","name","credit_card_default"],axis=1)

In [ ]:
def objective(trial: Trial,X,y) -> float:
    
    joblib.dump(study, 'study.pkl')

    param = {
                "n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
                'max_depth':trial.suggest_int('max_depth', 2, 25),
                'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
                'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
                'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
                'gamma':trial.suggest_int('gamma', 0, 5),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
                'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
                'nthread' : -1
            }
    
    model = xgb.XGBClassifier(**param)
    
    return cross_val_score(model,X_train,y_train, cv=3).mean()

In [ ]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
study.optimize(lambda trial : objective(trial,X_train,y_train),n_trials= 50)

In [ ]:
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [37]:
##Geting the categorical and Numberical values
cat_cols1= X1.select_dtypes(exclude="number").columns
num_cols1= X1.select_dtypes(include="number").columns

In [38]:
cat_cols1

Index(['occupation_type'], dtype='object')

In [39]:
num_cols1

Index(['age', 'no_of_children', 'net_yearly_income', 'no_of_days_employed',
       'total_family_members', 'migrant_worker', 'yearly_debt_payments',
       'credit_limit', 'credit_limit_used(%)', 'credit_score', 'prev_defaults',
       'default_in_last_6months'],
      dtype='object')

In [40]:
from sklearn.preprocessing import RobustScaler

In [43]:
RScaler=RobustScaler()
num_features=RScaler.fit_transform(X1[num_cols1])

In [44]:
num_features

array([[ 0.4375    ,  0.        , -0.55821835, ..., -1.48466258,
         2.        ,  1.        ],
       [-0.625     ,  0.        , -0.54133917, ...,  0.43558282,
         0.        ,  0.        ],
       [-0.125     ,  0.        ,  0.51145914, ..., -0.83435583,
         0.        ,  0.        ],
       ...,
       [-0.75      ,  0.        ,  0.77313911, ..., -0.36196319,
         0.        ,  0.        ],
       [-0.4375    ,  0.        , -0.12560941, ...,  0.11656442,
         0.        ,  0.        ],
       [-0.0625    ,  1.        ,  1.27064595, ..., -0.63803681,
         0.        ,  0.        ]])

In [47]:
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()

In [48]:
cat_features=le.fit_transform(X1[cat_cols1])

c:\programdata\anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [49]:
cat_features

array([17,  8,  8, ...,  3,  8, 17])

### Preprocessing Pipeline 

In [53]:
#Preprocessing Pipelines for numerical and categorial values
categorical_pipeline = Pipeline(
    steps=[
        ("Label",LabelEncoder())
    ]
)

numeric_pipeline = Pipeline(
    steps=[ 
           ("scale", RobustScaler())]
)

In [54]:
from sklearn.compose import ColumnTransformer

full_processor =FeatureUnion([("pca", PCA(n_components=1)),("svd", TruncatedSVD(n_components=2))])

In [55]:
train_pre=full_processor.fit_transform(X1)

TypeError: fit_transform() takes 2 positional arguments but 3 were given